In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
df_train = pd.read_csv('../data/train_cleaned.csv')
df_test = pd.read_csv('../data/test_cleaned.csv')

df_train.head()

,Unnamed: 0,Unnamed: 0.1,title,address,property_name,property_type,num_beds,num_baths,size_sqft,furnishing,lat,lng,subzone,planning_area,price,built_year,tenure,available_unit_types
0,0,0,hdb flat for sale in 506b serangoon north aven...,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,4,2,1575,unspecified,1.373047,103.8750,serangoon north,serangoon,995400.0,1992,99-year leasehold,"1, 2, 3, 4, 5, 6 br"
1,1,1,4 bed condo for sale in meyerhouse,128 meyer road,meyerhouse,condo,4,6,3070,partial,1.298828,103.8750,mountbatten,marine parade,8485000.0,2022,freehold,"studio, 3, 4, 5, 6 br"
2,2,2,3 bed condo for sale in leedon green,26 leedon heights,leedon green,condo,3,2,958,partial,1.312500,103.8125,farrer court,bukit timah,2626000.0,2023,freehold,"studio, 1, 2, 3, 4 br"
3,3,3,2 bed condo for sale in one bernam,1 bernam street,one bernam,condo,2,1,732,unspecified,1.274414,103.8125,anson,downtown core,1764000.0,2026,99-year leasehold,"studio, 1, 2, 3, 4, 5 br"
4,4,4,2 bed condo for sale in forett @ bukit timah,32 toh tuck road,forett @ bukit timah,condo,2,2,689,unspecified,1.338867,103.7500,bukit batok south,bukit batok,1702000.0,2024,freehold,"studio, 1, 2, 3, 4, 5 br"


In [3]:
df_train = df_train.drop(['Unnamed: 0.1', 'Unnamed: 0', 'title','address','available_unit_types'], axis=1)
df_train.head()

,property_name,property_type,num_beds,num_baths,size_sqft,furnishing,lat,lng,subzone,planning_area,price,built_year,tenure
0,hdb-serangoon estate,hdb,4,2,1575,unspecified,1.373047,103.8750,serangoon north,serangoon,995400.0,1992,99-year leasehold
1,meyerhouse,condo,4,6,3070,partial,1.298828,103.8750,mountbatten,marine parade,8485000.0,2022,freehold
2,leedon green,condo,3,2,958,partial,1.312500,103.8125,farrer court,bukit timah,2626000.0,2023,freehold
3,one bernam,condo,2,1,732,unspecified,1.274414,103.8125,anson,downtown core,1764000.0,2026,99-year leasehold
4,forett @ bukit timah,condo,2,2,689,unspecified,1.338867,103.7500,bukit batok south,bukit batok,1702000.0,2024,freehold


In [4]:
df_test = df_test.drop(['Unnamed: 0.1', 'Unnamed: 0', 'title','address','available_unit_types'], axis=1)
df_test.head()

,property_name,property_type,num_beds,num_baths,size_sqft,furnishing,lat,lng,subzone,planning_area,built_year,tenure
0,the gazania,condo,1,1,463,unfurnished,1.344727,103.8750,upper paya lebar,serangoon,2022,freehold
1,vue 8 residence,condo,3,3,1033,unspecified,1.379883,103.9375,pasir ris west,pasir ris,2017,99-year leasehold
2,icon,condo,1,1,570,fully,1.294922,103.8750,bras basah,museum,2007,99-year leasehold
3,keat hong colours,hdb,3,2,1216,unspecified,1.373047,103.7500,keat hong,choa chu kang,2017,99-year leasehold
4,toa payoh spring,hdb,3,2,936,unspecified,1.341797,103.8750,braddell,toa payoh,1973,99-year leasehold


In [5]:
print(df_train.shape)
print(df_test.columns)

(17219, 13)
Index(['property_name', 'property_type', 'num_beds', 'num_baths', 'size_sqft',
       'furnishing', 'lat', 'lng', 'subzone', 'planning_area', 'built_year',
       'tenure'],
      dtype='object')


In [6]:
#Calculate price per sqft of every row
df_train["price_per_sqft"] = df_train["price"]/df_train["size_sqft"] 
df_train["price_per_sqft"] = df_train["price_per_sqft"].round(2)


In [7]:
#Target encode Property Name
prop_name_agg = df_train.groupby('property_name').agg('mean')
df_train = df_train.merge(prop_name_agg["price_per_sqft"], on=['property_name'], how='left')
df_train.rename(columns={"price_per_sqft_y": "property_name_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)
#df_train.drop(["property_name", "available_unit_types"], axis=1, inplace=True)
df_train.head()
'''
df_test = df_test.merge(prop_name_agg["price_per_sqft"], on=['property_name'], how='left')
df_test.rename(columns={"price_per_sqft_y": "property_name_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)
df_test.drop(["property_name", "available_unit_types"], axis=1, inplace=True)
'''

'\ndf_test = df_test.merge(prop_name_agg["price_per_sqft"], on=[\'property_name\'], how=\'left\')\ndf_test.rename(columns={"price_per_sqft_y": "property_name_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)\ndf_test.drop(["property_name", "available_unit_types"], axis=1, inplace=True)\n'

In [8]:
#Target encode Address
'''
address_agg = df_train.groupby('address').agg('mean')
df_train = df_train.merge(address_agg["price_per_sqft"], on=['address'], how='left')
df_train.rename(columns={"price_per_sqft_x": "price_per_sqft"}, inplace=True)
df_train.rename(columns={"price_per_sqft_y": "address_encoded"}, inplace=True)
#df_train.drop(["address"], axis=1, inplace=True)

df_test = df_test.merge(address_agg["price_per_sqft"], on=['address'], how='left')
df_test.rename(columns={"price_per_sqft_y": "address_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)
df_test.drop(["address"], axis=1, inplace=True)
'''

'\naddress_agg = df_train.groupby(\'address\').agg(\'mean\')\ndf_train = df_train.merge(address_agg["price_per_sqft"], on=[\'address\'], how=\'left\')\ndf_train.rename(columns={"price_per_sqft_x": "price_per_sqft"}, inplace=True)\ndf_train.rename(columns={"price_per_sqft_y": "address_encoded"}, inplace=True)\n#df_train.drop(["address"], axis=1, inplace=True)\n\ndf_test = df_test.merge(address_agg["price_per_sqft"], on=[\'address\'], how=\'left\')\ndf_test.rename(columns={"price_per_sqft_y": "address_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)\ndf_test.drop(["address"], axis=1, inplace=True)\n'

In [9]:
#Target encode Subzone
subzone_agg = df_train.groupby('subzone').agg('mean')
df_train = df_train.merge(subzone_agg["price_per_sqft"], on=['subzone'], how='left')
df_train.rename(columns={"price_per_sqft_x": "price_per_sqft"}, inplace=True)
df_train.rename(columns={"price_per_sqft_y": "subzone_encoded"}, inplace=True)
#df_train.drop(["subzone"], axis=1, inplace=True)
'''
df_test = df_test.merge(subzone_agg["price_per_sqft"], on=['subzone'], how='left')
df_test.rename(columns={"price_per_sqft_y": "subzone_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)
df_test.drop(["subzone"], axis=1, inplace=True)
'''

'\ndf_test = df_test.merge(subzone_agg["price_per_sqft"], on=[\'subzone\'], how=\'left\')\ndf_test.rename(columns={"price_per_sqft_y": "subzone_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)\ndf_test.drop(["subzone"], axis=1, inplace=True)\n'

In [10]:
#Target encode planning Area
subzone_agg = df_train.groupby('planning_area').agg('mean')
df_train = df_train.merge(subzone_agg["price_per_sqft"], on=['planning_area'], how='left')
df_train.rename(columns={"price_per_sqft_x": "price_per_sqft"}, inplace=True)
df_train.rename(columns={"price_per_sqft_y": "planning_area_encoded"}, inplace=True)
#df_train.drop(["planning_area"], axis=1, inplace=True)

'''
df_test = df_test.merge(subzone_agg["price_per_sqft"], on=['planning_area'], how='left')
df_test.rename(columns={"price_per_sqft_y": "planning_area_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)
df_test.drop(["planning_area"], axis=1, inplace=True)
'''

'\ndf_test = df_test.merge(subzone_agg["price_per_sqft"], on=[\'planning_area\'], how=\'left\')\ndf_test.rename(columns={"price_per_sqft_y": "planning_area_encoded", "price_per_sqft_x" : "price_per_sqft"}, inplace=True)\ndf_test.drop(["planning_area"], axis=1, inplace=True)\n'

In [11]:
#Label Encode property_type
pd.set_option("display.max_columns", 15)
type_to_enum_map = {t: idx+1 for idx, t in enumerate(sorted(df_train.property_type.unique()))}
df_train.property_type = df_train.property_type.replace(type_to_enum_map,).astype(np.int16)
df_train.head()

df_test.property_type = df_test.property_type.replace(type_to_enum_map,).astype(np.int16)


In [12]:
#steps from here on are what we need to do in order to calculate the psf price feature for a dataset

from math import cos, radians

def get_lat_offset(distance_in_m):
    offset = distance_in_m/111111
    return offset

def get_lng_offset(distance_in_m, lat):
    offset = distance_in_m/(111111*cos(radians(lat)))
    return offset

def get_bounding_square(lat, lng, side):
    lat_offset = get_lat_offset(side/2)
    lng_offset = get_lng_offset(side/2, lat)
    
    top_lat = lat + lat_offset
    bottom_lat = lat - lat_offset
    
    right_lng = lng + lng_offset
    left_lng = lng - lng_offset
    
    return top_lat, bottom_lat, right_lng, left_lng

bbox_side = 1000

def get_average_price_psf_within_bbox(df,lat, lng):
    bbox = get_bounding_square(lat, lng, bbox_side)
    filtered_df = df[(df.lat < bbox[0]) & (df.lat > bbox[1]) & (df.lng < bbox[2]) & (df.lng > bbox[3])]
    return filtered_df['price_per_sqft'].mean()
    
df_train = df_train[df_train.price > 0]
df_train = df_train[df_train.lat<=1.47]
df_train = df_train[df_train.size_sqft>0]

df_train['price_psf_lat_lng'] = df_train.apply(lambda x: get_average_price_psf_within_bbox(df_train,x['lat'], x['lng']), axis=1)


In [13]:
#One hot encode furnishing
x = pd.get_dummies(df_train.furnishing)
df_train['furnishing_unspecified'] = x.unspecified.astype(int)
df_train['furnishing_partial'] = x.partial.astype(int)
df_train['furnishing_unfurnished'] = x.unfurnished.astype(int)
df_train['furnishing_fully'] = x.fully.astype(int)
df_train.drop(columns=['furnishing'], axis = 1, inplace=True)


df_test['furnishing_unspecified'] = x.unspecified.astype(int)
df_test['furnishing_partial'] = x.partial.astype(int)
df_test['furnishing_unfurnished'] = x.unfurnished.astype(int)
df_test['furnishing_fully'] = x.fully.astype(int)
df_test.drop(columns=['furnishing'], axis = 1, inplace=True)

df_train.head()


,property_name,property_type,num_beds,num_baths,size_sqft,lat,lng,...,subzone_encoded,planning_area_encoded,price_psf_lat_lng,furnishing_unspecified,furnishing_partial,furnishing_unfurnished,furnishing_fully
0,hdb-serangoon estate,8,4,2,1575,1.373047,103.8750,...,626.643333,1913.641513,925.362012,1,0,0,0
1,meyerhouse,4,4,6,3070,1.298828,103.8750,...,2474.825711,2317.931757,2667.460626,0,1,0,0
2,leedon green,4,3,2,958,1.312500,103.8125,...,2864.480273,2579.110087,2827.125707,0,1,0,0
3,one bernam,4,2,1,732,1.274414,103.8125,...,2845.122957,2979.342684,1957.168229,1,0,0,0
4,forett @ bukit timah,4,2,2,689,1.338867,103.7500,...,1808.754700,1328.452759,1719.206812,1,0,0,0


In [14]:
#One hot encode tenure

pd.set_option("display.max_columns", 25)

x = pd.get_dummies(df_train.tenure)
#print(x)
df_train['tenure_99'] = x['99-year leasehold'].astype(int)
df_train['tenure_999'] = x['999-year leasehold'].astype(int)
df_train['tenure_freehold'] = x['freehold'].astype(int)
df_train.drop(columns=['tenure'], axis = 1, inplace=True)

df_test['tenure_99'] = x['99-year leasehold'].astype(int)
df_test['tenure_999'] = x['999-year leasehold'].astype(int)
df_test['tenure_freehold'] = x['freehold'].astype(int)
df_test.drop(columns=['tenure'], axis = 1, inplace=True)


In [15]:
#Built Year
df_train.built_year = 2022 - df_train.built_year
df_test.built_year = 2022 - df_test.built_year

df_train.head()


,property_name,property_type,num_beds,num_baths,size_sqft,lat,lng,subzone,planning_area,price,built_year,price_per_sqft,property_name_encoded,subzone_encoded,planning_area_encoded,price_psf_lat_lng,furnishing_unspecified,furnishing_partial,furnishing_unfurnished,furnishing_fully,tenure_99,tenure_999,tenure_freehold
0,hdb-serangoon estate,8,4,2,1575,1.373047,103.8750,serangoon north,serangoon,995400.0,30,632.00,811.803488,626.643333,1913.641513,925.362012,1,0,0,0,1,0,0
1,meyerhouse,4,4,6,3070,1.298828,103.8750,mountbatten,marine parade,8485000.0,0,2763.84,2812.763529,2474.825711,2317.931757,2667.460626,0,1,0,0,0,0,1
2,leedon green,4,3,2,958,1.312500,103.8125,farrer court,bukit timah,2626000.0,-1,2741.13,2946.238404,2864.480273,2579.110087,2827.125707,0,1,0,0,0,0,1
3,one bernam,4,2,1,732,1.274414,103.8125,anson,downtown core,1764000.0,-4,2409.84,2880.462661,2845.122957,2979.342684,1957.168229,1,0,0,0,1,0,0
4,forett @ bukit timah,4,2,2,689,1.338867,103.7500,bukit batok south,bukit batok,1702000.0,-2,2470.25,2238.394476,1808.754700,1328.452759,1719.206812,1,0,0,0,0,0,1


In [16]:
price_per_sqft = []
count=0
for i in range(len(df_test)):
    name = df_test.loc[i, "property_name"]
    lat = df_test.loc[i, "lat"]
    lng = df_test.loc[i, "lng"]
    sub = df_test.loc[i, "subzone"]
    area = df_test.loc[i, "planning_area"]
    if name in df_train['property_name'].values:
        price_per_sqft.append(df_train[df_train['property_name'] == name]['property_name_encoded'].values[0])
    elif lat in df_train['lat'].values:
        df_lat = df_train[df_train.lat==lat]
        if lng in df_lat['lng'].values:
            price_per_sqft.append(df_lat[df_lat['lng'] == lng]['price_psf_lat_lng'].values[0]) 
        else:
            price_per_sqft.append(1)
    elif sub in df_train['subzone'].values:
        price_per_sqft.append(df_train[df_train['subzone'] == name]['subzone_encoded'].values[0])
    elif sub in df_train['planning_area'].values:
        price_per_sqft.append(df_train[df_train['planning_area'] == name]['planning_area_encoded'].values[0])
    else:
        price_per_sqft.append(1)
 
            

In [18]:
df_test['price_per_sqft'] = price_per_sqft
df_test = df_test[df_test.price_per_sqft > 2] 

In [19]:
df_train = df_train.drop(['property_name','lat', 'lng', 'subzone', 'planning_area',
                          'property_name_encoded',
       'subzone_encoded', 'planning_area_encoded', 'price_psf_lat_lng'], axis=1)


In [20]:
df_test = df_test.drop(['property_name','lat', 'lng', 'subzone', 'planning_area'], axis=1)

In [21]:
df_train.to_csv('../data/train_modelling.csv')
df_test.to_csv('../data/test_modelling.csv')

In [22]:
df_train.shape

(17219, 14)

In [23]:
df_test.shape

(5900, 13)